In [2]:
import cv2
import mediapipe as mp
import os
from datetime import datetime

# Dictionary of selected facial landmark indices
selected_indices = {
    "Eyebrows": [21, 22, 23, 24],
    "Eyes": [36, 39, 41, 42, 45, 47],
    "Nose": [30, 31, 35],
    "Mouth": [48, 51, 54, 55, 61],
    "Jawline": [1, 9]
}

# Initialize MediaPipe Face Mesh
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(min_detection_confidence=0.5, min_tracking_confidence=0.5)

# Initialize OpenCV Video Capture
cap = cv2.VideoCapture(0)

# Create a folder to save captured images if not exists
save_folder = "Captured_Images"
os.makedirs(save_folder, exist_ok=True)

while cap.isOpened():
    ret, frame = cap.read()
    
    if not ret:
        break
    
    # Flip the frame horizontally for better viewing
    frame = cv2.flip(frame, 1)
    
    # Convert the frame to RGB for face mesh processing
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    # Process the frame to detect face landmarks
    results = face_mesh.process(rgb_frame)
    
    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            for category, indices in selected_indices.items():
                for idx in indices:
                    # Get the coordinates of the landmark
                    h, w, _ = frame.shape
                    x = int(face_landmarks.landmark[idx].x * w)
                    y = int(face_landmarks.landmark[idx].y * h)
                    
                    # Draw a circle around the selected landmarks
                    cv2.circle(frame, (x, y), 2, (0, 255, 0), -1)
    
    # Display the frame with the landmarks
    cv2.imshow("Live Face Landmarks", frame)
    
    # Capture the image if 'c' is pressed
    key = cv2.waitKey(1) & 0xFF
    if key == ord('c'):
        # Save the current frame with a timestamped filename
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = os.path.join(save_folder, f"captured_{timestamp}.jpg")
        cv2.imwrite(filename, frame)
        print(f"Image captured and saved as {filename}")
    
    # Break the loop if 'q' is pressed
    if key == ord('q'):
        break

# Release the camera and close the window
cap.release()
cv2.destroyAllWindows()


Image captured and saved as Captured_Images\captured_20241206_103618.jpg
Image captured and saved as Captured_Images\captured_20241206_103622.jpg
Image captured and saved as Captured_Images\captured_20241206_103626.jpg


In [9]:
import cv2
import mediapipe as mp
import numpy as np
import joblib
from scipy.spatial import distance
from sklearn.preprocessing import StandardScaler

# Load the trained model and scaler
model = joblib.load('stress_detection_model_distance_05.pkl')
scaler = joblib.load('scaler_05.pkl')

# Initialize Mediapipe FaceMesh
mp_face_mesh = mp.solutions.face_mesh
mp_drawing = mp.solutions.drawing_utils
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=False, max_num_faces=1)

# Define landmarks to extract
selected_indices = {
    "Eyebrows": [21, 22, 23, 24],
    "Eyes": [36, 39, 41, 42, 45, 47],
    "Nose": [30, 31, 35],
    "Mouth": [48, 51, 54, 55, 61],
    "Jawline": [1, 9]
}
all_indices = [idx for indices in selected_indices.values() for idx in indices]

# Define pairs of landmarks for distance calculation
distance_pairs = [
    (21, 22),  # Eyebrow width
    (36, 39),  # Left eye width
    (42, 45),  # Right eye width
    (30, 48),  # Nose to mouth corner (left)
    (30, 54),  # Nose to mouth corner (right)
    (48, 54),  # Mouth width
    (1, 9),    # Jawline height
    (51, 61),  # Vertical mouth opening
    (21, 23)   # Vertical eyebrow distance
]

# Initialize webcam
cap = cv2.VideoCapture(0)

# Define landmarks drawing function
def draw_landmarks(frame, face_landmarks):
    for feature, indices in selected_indices.items():
        for i in indices:
            if face_landmarks.landmark[i].visibility > 0:  # Ensure the landmark is visible
                # Draw the landmark with blue color for specific facial features
                mp_drawing.draw_landmarks(
                    frame, face_landmarks, [i],
                    mp_drawing.DrawingSpec(color=(0, 0, 255), thickness=2, circle_radius=2),  
                    mp_drawing.DrawingSpec(color=(0, 0, 255), thickness=2, circle_radius=2)   
                )

    # Draw all contours with the original color scheme
    mp_drawing.draw_landmarks(
        frame, face_landmarks, mp_face_mesh.FACEMESH_CONTOURS,
        mp_drawing.DrawingSpec(color=(80, 110, 10), thickness=1, circle_radius=1),
        mp_drawing.DrawingSpec(color=(80, 256, 121), thickness=1, circle_radius=1)
    )

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Flip the frame horizontally for a more natural selfie view
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Process the frame with FaceMesh
    results = face_mesh.process(frame_rgb)

    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            # Extract 2D landmarks
            landmark_coords = []
            for idx in all_indices:
                x = face_landmarks.landmark[idx].x
                y = face_landmarks.landmark[idx].y
                landmark_coords.append((x, y))

            # Calculate inter-pupillary distance for normalization
            left_eye = landmark_coords[all_indices.index(36)]
            right_eye = landmark_coords[all_indices.index(42)]
            inter_pupillary_dist = distance.euclidean(left_eye, right_eye)

            # Calculate distances
            distances = []
            for pair in distance_pairs:
                point1 = landmark_coords[all_indices.index(pair[0])]
                point2 = landmark_coords[all_indices.index(pair[1])]
                dist = distance.euclidean(point1, point2)
                normalized_dist = dist / inter_pupillary_dist  # Normalize distance
                distances.append(normalized_dist)

            # Calculate additional ratios as features
            mouth_width = distances[distance_pairs.index((48, 54))]
            mouth_height = distances[distance_pairs.index((51, 61))]
            if mouth_height > 0:  # Avoid division by zero
                mouth_ratio = mouth_width / mouth_height
            else:
                mouth_ratio = 0

            eyebrow_width = distances[distance_pairs.index((21, 22))]
            eyebrow_height = distances[distance_pairs.index((21, 23))]
            if eyebrow_height > 0:
                eyebrow_ratio = eyebrow_width / eyebrow_height
            else:
                eyebrow_ratio = 0

            # Append ratios and distances for prediction
            features = distances + [mouth_ratio, eyebrow_ratio]

            # Scale the features before prediction
            features_scaled = scaler.transform([features])

            # Predict the stress level using the trained model
            prediction = model.predict(features_scaled)

            # Draw landmarks on the frame
            draw_landmarks(frame, face_landmarks)

            # Display the prediction
            stress_label = "Stress" if prediction == 1 else "Non-Stress"
            cv2.putText(frame, f"Prediction: {stress_label}", (30, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)

    else:
        # If no face detected, display message
        cv2.putText(frame, "No Face Detected", (30, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)

    # Show the frame with landmarks and prediction
    cv2.imshow('Face Stress Detection', frame)

    # Exit the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture and close windows
cap.release()
cv2.destroyAllWindows()


c:\Users\Kamesh\AppData\Local\Programs\Python\Python310\lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


ValueError: X has 11 features, but RandomForestClassifier is expecting 4 features as input.